In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np
from PIL import Image
from pathlib import Path
import os
from torchvision import models

In [35]:
class catDogDataset(torch.utils.data.Dataset):
    def __init__(self, rootFolder):
        self.rootFolder = rootFolder
        self.paths = []
        self.labels = []
        self.getImgsAndLabels()

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self,  index):
        image = self.paths[index]
        return self.reformat(image), self.labels[index]
    
    def read_image(self, path: Path):
        """This function read an image from a path.
        The read is perform using PIL.Image (cause PyTorch).
        """

        image = Image.open(path)
        if image is None:
            raise ValueError(f'Cannot read image {path}.')
        image = image.convert('RGB')
        return np.asarray(image)
    
    def getImgsAndLabels(self):
        
        current_dir = Path.cwd()
        par_dir = current_dir.parent
        dir = par_dir / self.rootFolder
        
        for class_dir in dir.iterdir():
            if class_dir.is_dir():
                # Get the folder name as the class label    
                if class_dir.name == "Cat":
                    class_label = 1
                else: 
                    class_label = 0 

                # Iterate over the image files in the class folder
                for image_path in class_dir.glob('*.jpg'):
                    self.paths.append(image_path)
                    self.labels.append(class_label)

    

    def reformat(self, img_path):
        '''This function take in a image path and return it as a (1, 28, 28) tensor to pass into the NN Model
        ,
        Attribute:
        ---------
        img_path : the path to the image on the computer
        '''
        img = self.read_image(img_path) # H, W, C
        #resize to 224x224
        img_ = cv.resize(img, (224, 224))

        #reshape 
        if img_.shape == (224, 224):
            img_= cv.cvtColor(img_,cv.COLOR_GRAY2RGB)
        # #get the color
        # col = img_.shape[2]
        #convert to tensor
        img_ = torch.from_numpy(img_).reshape(3, 224, 224).float()

        return img_

In [27]:

print(dir)


c:\VGG16\Data\train


In [41]:
train_set = catDogDataset(r"Data/train")
test_set = catDogDataset(r"Data/test")
val_set = catDogDataset(r"Data/val")

In [38]:
train_dataLoader = DataLoader(dataset = train_set,
                        batch_size = 70, 
                        shuffle= True,
                        drop_last= True)
for picture, label in train_dataLoader:
    print(f"Shape of X [N, C, H, W]: {picture.shape}")
    print(f"Shape of y: {label.shape} {label.dtype}")
    break
len(train_dataLoader)

test_dataLoader = DataLoader(dataset = test_set,
                             batch_size = 70,
                             shuffle= True,
                             drop_last= True)
val_dataLoader = DataLoader(dataset = val_set,
                            batch_size = 70,
                            shuffle= True,
                            drop_last= True)

Shape of X [N, C, H, W]: torch.Size([70, 3, 224, 224])
Shape of y: torch.Size([70]) torch.int64


In [40]:
train_set.__getitem__(28)[0].shape

torch.Size([3, 224, 224])

In [2]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [20]:
vgg = models.vgg16(pretrained =True)


c:\VGG16\venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\VGG16\venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\ASUS/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100.0%


In [21]:
vgg.classifier[6] = nn.Linear(in_features=4096, out_features=2)
vgg.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [22]:
params_to_update = []
update_params_name = ['classifier.6.weight', 'classifier.6.bias']
for name, param in vgg.named_parameters():
    if name in update_params_name:
        param.requires_grad = True
        params_to_update.append(param)
        print(name)
    else:
        param.requires_grad = False

classifier.6.weight
classifier.6.bias


In [23]:
loss_fun = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = params_to_update, lr=1e-3, momentum=0.9)

In [24]:
def train(dataloader, model, loss_fun, optimizer):
    model.train()
    num_batches = len(dataloader)
    epoch_loss = 0
    for batch, (pic, lab) in enumerate(dataloader):
        pic, lab = pic.to(device), lab.to(device)
        
        # Compute prediction error
        pred = model(pic)
        loss = loss_fun(pred, lab)

        # Backpropagation
        optimizer.zero_grad() #clear the model's gradient to avoid gradient accumulation
        loss.backward() #compute the loss function's gradients
        optimizer.step() #update the parameters

         #Calculate loss
        epoch_loss = epoch_loss + loss.item()

    avg_loss = epoch_loss / num_batches
    print("Epoch's avg train loss:", avg_loss)
    return avg_loss


def test_validate(dataloader, model, loss_fun):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for pic, lab in dataloader:
            pic, lab = pic.to(device), lab.to(device)
            pred = model(pic)
            test_loss += loss_fun(pred, lab).item() #get the loss
            correct += (pred.argmax(1) == lab).type(torch.float).sum().item() #get how many times the model guess correctuly
    test_loss /= num_batches #loss per batch
    correct /= size #accuracy
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return test_loss

In [25]:
epochs = 5
for epoch in range(epochs):
    print("Rpoch:", epoch + 1)
    epoch_train_loss = train(train_dataLoader, vgg, loss_fun, optimizer)
    epoch_val_loss = test_validate(val_dataLoader, vgg, loss_fun)

Rpoch: 1


c:\VGG16\venv\lib\site-packages\PIL\TiffImagePlugin.py:864: UserWarning: Truncated File Read
  warnings.warn(str(msg))


KeyboardInterrupt: 